In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from pymongo import MongoClient
client = MongoClient()
import pandas as pd
import time

In [ ]:
driver = webdriver.Chrome()

In [ ]:
# https://bu.dauphine.psl.eu
# Trouver -> Economie
# EconLit with Full Text
# prenom nom -> Field Author (AU)
# Publication Type - Journal Article
# 1996 - 2019
# recopier le lien sur la première page

In [ ]:
## A changer manuellment à chaque fois
AUTHOR_NAME_SIOE = 'MENARD'

#### FAIRE LA RECHERCHE CI-DESSUS AVANT d'EXECUTER CI-DESSOUS

In [ ]:
MAIN_URL = driver.current_url

In [ ]:
db = client['sioe_articles']
articles = db.sioe_articles

In [ ]:
sanitize_key = lambda x: x.lower().replace(' ', '_').replace(':', '')

In [ ]:
# Détermine le nombre de pages
links_to_pages = driver.find_elements_by_xpath('//ul[@class="results-paging nav-list"]/li/a')
links_to_pages = [l.text for l in links_to_pages if l.text not in ['Previous', 'Next']]
page_max = 1 if len(links_to_pages) == 0 else int(max(links_to_pages))

In [ ]:
# Datas d'un article vers un document JSON
def data_to_json(datas):
    dict_out = dict()
    for d in datas:
        key = d[0].text
        value = d[1].text
        dict_out[key] = value
    dict_out['Title'] = dict_out.pop('') # Car pas de label pour le titre
    dict_out['Author_SIOE'] = AUTHOR_NAME_SIOE
    return dict_out

In [ ]:
# Insère tous les articles d'une page dans MongoDb
def insert_data_to_mongo_from_articles_links_in_one_page():
    links_articles_page = [l.get_attribute('href') for l in driver.find_elements_by_xpath('//a[@class="title-link color-p4"]')]
    for l in links_articles_page:    
        driver.get(l)
        values = driver.find_elements_by_xpath('//dd[@data-auto="citation_field_value"]')
        labels = driver.find_elements_by_xpath('//dt[@data-auto="citation_field_label"]')
        datas = list(zip(labels, values))
        articles.insert_one(data_to_json(datas))
        time.sleep(3)

### Exécussion du programme

In [ ]:
# insère la première page
insert_data_to_mongo_from_articles_links_in_one_page()
# Si plus d'une page, alors insère les pages suivantes
if page_max > 1:
    for page_nb in range(1, page_max): # la prmière page est à 0, la deuxième à 1, etc.
        driver.get(MAIN_URL) # retourne à la page principale
        link = 'ctl00_ctl00_MainContentArea_MainContentArea_bottomMultiPage_rptPageLinks_ctl0{}_lnkPageLink'.format(page_nb)
        driver.find_element_by_id(link).click() # clique le lien vers la page
        insert_data_to_mongo_from_articles_links_in_one_page()